In [1]:
import numpy as np
import pandas as pd

from scipy.io import loadmat
from scipy.special import expit
from scipy.optimize import minimize

from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = loadmat('../input/andrew_ng/ex4data1.mat')
weights = loadmat('../input/andrew_ng/ex4weights.mat')

X, y = data['X'], data['y']
theta1, theta2 = weights['Theta1'], weights['Theta2']

X.shape, y.shape, theta1.shape, theta2.shape

((5000, 400), (5000, 1), (25, 401), (10, 26))

In [3]:
# there are many ways to encode, I used the pandas's get_dummies
y_enc = pd.get_dummies(y.flatten()).values
y_enc.shape

(5000, 10)

In [4]:
l1_size, l2_size, l3_size = 400, 25, 10

all_theta = np.concatenate((theta1.ravel(), theta2.ravel())).flatten()
all_theta.shape

(10285,)

In [5]:
def sigmoid(z):
    return expit(z)

In [6]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1-sigmoid(z)))

In [7]:
def forward_progagation(X, theta1, theta2):
    a1 = np.c_[np.ones((X.shape[0], 1)), X]

    z2 = a1.dot(theta1.T)
    a2 = sigmoid(z2)
    a2 = np.c_[np.ones((a2.shape[0], 1)), a2]

    z3 = a2.dot(theta2.T)
    a3 = sigmoid(z3)

    return a1, z2, a2, z3, a3

In [8]:
# Accuracy
np.mean(np.argmax(forward_progagation(X, theta1, theta2)[-1], axis=1)+1==y.ravel())*100

97.52

In [9]:
def cost(all_theta, l1_size, l2_size, l3_size, X, y, reg):    
    cost = 0
    m = X.shape[0]
    theta1 = all_theta[:(l1_size+1)*l2_size].reshape(l2_size, (l1_size+1))
    theta2 = all_theta[(l1_size+1)*l2_size:].reshape(l3_size, (l2_size+1))
    y = y.ravel().reshape(-1,10)
    hx = forward_progagation(X, theta1, theta2)[-1]
    
    for i in range(m):
        a = y[i, :].dot(np.log(hx[i, :]))
        b = (1-y[i, :]).dot(np.log((1-hx[i, :])))
        cost -= a+b
    return cost/m + (reg/(2*m))*(np.sum(np.power(theta1,2)) + np.sum(np.power(theta2,2)))
    # comment the regularization part to see result without regularization

cost(all_theta, l1_size, l2_size, l3_size, X, y_enc, 1)

0.3844877962428938

In [10]:
def back_propagation(all_theta, l1_size, l2_size, l3_size, X, y, reg):
    m = X.shape[0]

    theta1 = all_theta[:(l1_size+1)*l2_size].reshape(l2_size, (l1_size+1))
    theta2 = all_theta[(l1_size+1)*l2_size:].reshape(l3_size, (l2_size+1))

    a1, z2, a2, z3, h = forward_progagation(X, theta1, theta2)
    D3 = np.zeros(theta2.shape)  # 10X26
    D2 = np.zeros(theta1.shape)  #25X401

#     J = cost(all_theta, l1_size, l2_size, l3_size, X, y, 1)

    for t in range(m):
        yt = h[t] - y[t]  #1X10
        d2 = np.multiply(yt.dot(theta2), a2[t])  #1X26

        D3 += yt.reshape(1, -1).T.dot(a2[t].reshape(1, -1))
        D2 += d2.reshape(1, -1).T[1:].dot(a1[t].reshape(1, -1))
    D3 /= m
    D2 /= m
    D3 = np.concatenate((D3[:, :1], theta2[:, 1:] * (reg/m)), axis=1)
    D2 = np.concatenate((D2[:, :1], theta1[:, 1:] * (reg/m)), axis=1)

    grad = np.concatenate((np.ravel(D2), np.ravel(D3)), axis=0)
#     print(np.unique(grad.flatten()==all_theta.flatten()))
    return grad.flatten()

back_propagation(all_theta, l1_size, l2_size, l3_size, X, y_enc, 1)

array([-8.43553018e-03, -2.11248326e-12,  4.38829369e-13, ...,
       -4.95591575e-05,  2.56018236e-04, -2.65504084e-04])

In [11]:
optmz = minimize(fun=back_propagation, x0=all_theta.reshape(all_theta.shape[0],1), args=(l1_size, l2_size, l3_size, X, y_enc, 1), jac=True, options={'maxiter': 250})
optmz

      fun: -0.00843553018040702
 hess_inv: array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])
      jac: -2.1124832621366728e-12
  message: 'Optimization terminated successfully.'
     nfev: 1
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([-2.25623899e-02, -1.05624163e-08,  2.19414684e-09, ...,
       -2.47795788e-01,  1.28009118e+00, -1.32752042e+00])

In [12]:
optmz.keys()

dict_keys(['status', 'nit', 'nfev', 'message', 'hess_inv', 'success', 'fun', 'x', 'njev', 'jac'])

In [13]:
optmz.x.shape, optmz.jac.shape

((10285,), ())

In [14]:
t1 = optmz.x[:(l1_size+1)*l2_size].reshape(l2_size, (l1_size+1))
t2 = optmz.x[(l1_size+1)*l2_size:].reshape(l3_size, (l2_size+1))

resp = forward_progagation(X, t1, t2)

# Accuracy
np.mean(np.argmax(forward_progagation(X, t1, t2)[-1], axis=1)+1==y.ravel())*100

97.52